# Projet maison n° 3

In [39]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [40]:
import requests
import zipfile
URL = "https://www.ssa.gov/oact/babynames/names.zip"
response = requests.get(URL)
open("name.zip", "wb").write(response.content)


with zipfile.ZipFile("name.zip","r") as zip_ref:
    zip_ref.extractall("./")

In [41]:
# names us
def df_names_us():
    df = pd.DataFrame()
    return df

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [42]:
import zipfile

import requests

def download_data(url, filename):
    try:
        response = requests.get(url)
        open(filename, "wb").write(response.content)
        return True
    except:
        return False

def unzip_data(filename):
    try:
        with zipfile.ZipFile(filename,"r") as zip_ref:
            zip_ref.extractall("./data/")
        return True
    except:
        return False


URL = "https://www.ssa.gov/oact/babynames/names.zip"
response = requests.get(URL)
open("us_name.zip", "wb").write(response.content)


with zipfile.ZipFile("us_name.zip","r") as zip_ref:
    zip_ref.extractall(r"./data/us_name")
    

french_data_url = "https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip"

download_data(french_data_url, "french_name.zip")
unzip_data("french_name.zip")

False

In [2]:
from os import listdir
from os.path import isfile, join

def list_file(path = "./data/us_name"):
    
    us_name_path = path
    files = [f for f in listdir(us_name_path) if isfile(join(us_name_path, f))]

    return files
len(list_file())

143

In [14]:
pd.read_csv('/home/gh7951/projects/telecom/h_ghalebi/Kit-Big-Data/session3/data/us_name/yob2017.txt', sep=",")

,Emma,F,19847
0,Olivia,F,18753
1,Ava,F,15998
2,Isabella,F,15222
3,Sophia,F,14910
4,Mia,F,13496
...,...,...,...
32642,Zykeem,M,5
32643,Zylin,M,5
32644,Zylis,M,5
32645,Zylyn,M,5


In [118]:

from cmath import nan
from unicodedata import name
from jack import Status
from nbformat import read
import pandas as pd
import re

from sqlalchemy import null
def read_files(file_name, us_name_path):
    year =re.findall(r'\d+', file_name)
    try:
        df = pd.read_csv(us_name_path+'/'+ file_name, sep=",",header=None, encoding = 'unicode_escape')
        df.columns = ['name', 'gender', 'births']
        
    
        
        df['year'] = year * len(df)
        
        return df , True
    except:
        data = {'name': None, 'gender': None, 'births': None, 'year': year}
        return pd.DataFrame(data) , False



def df_names_us(path = r"./data/us_name", columns_order = ["year", "name", "gender", "births"]):
    total= pd.DataFrame()

    length = 0
    for file in list_file(path=path):
        
        data,Status = read_files(file, path)
        if Status is not True:
            print (f'Filed to read {file}')
        length+= len(data)
        total = pd.concat([total, data], axis=0)
    #
    #total = remove_short_name(df=total, min_length=2)
    total = column_capitalize(total, col="name")
    #print(f" cumlative leghnt is {length} and total leghnt is {len(total)}")
    #total = total.dropna()

    #total =total.reset_index(drop=True)
    
        #  - Mêmes dtypes pour les colonnes
    total ['year'] = pd.to_datetime (total ['year'])
    total['year'] = pd.DatetimeIndex(total['year']).year
    total ['births'] = pd.to_numeric (total ['births'] )
    
    total = total.sort_values(by=[ 'gender', 'births','name','year'],
                                ascending=[False, False, True,False])
    total =total.reset_index()
    total = total.drop(columns=['index'])
    return total [columns_order]



def remove_short_name(df, min_length = 2):
    df = df[df.name.str.len() >= min_length]
    return df

def column_capitalize(datafram ,col = "name"):
    datafram[col] = datafram[col].str.capitalize()
    return datafram

# df1 = read_files(onlyfiles[0], us_name_path)
# df2 = read_files(onlyfiles[1], us_name_path)
# df3 = read_files(onlyfiles[2], us_name_path)
# all_row_pd = pd.concat([df1, df2, df3], axis=0)
# all_row_pd

us_data = df_names_us()

Filed to read NationalReadMe.pdf


In [ ]:
2052781

In [21]:
us_data

,year,name,gender,births
0,1947,James,M,94762
1,1957,Michael,M,92723
2,1947,Robert,M,91651
3,1956,Michael,M,90715
4,1958,Michael,M,90574
...,...,...,...,...
2052776,2014,Zyrihanna,F,5
2052777,2011,Zyrihanna,F,5
2052778,2010,Zyrihanna,F,5
2052779,2016,Zyriyah,F,5


In [22]:



us_data.columns.values.tolist() == ["year", "name", "gender", "births"]

True

In [23]:
len(us_data) - 2052781


0

In [20]:
us_data.loc[us_data.isnull().any(axis=1)].empty

True

In [24]:
us_data.index ==  pd.core.indexes.range.RangeIndex

array([False, False, False, ..., False, False, False])

In [8]:
us_data.index

Int64Index([ 6103,  7013,  6104,  6890,  7024,  6040,  6623,  6728,  6105,
             5684,
            ...
            19517, 20188, 17970, 19106, 17542, 19214, 19574, 19821, 18851,
            19215],
           dtype='int64', length=2052639)

In [15]:
pd.core.indexes.range.RangeIndex.start

In [119]:
# names fr

 #  - Mêmes valeurs pour la colonne "gender"
def convert_gender_label(number):
    if number == 1:
        return "M"
    else:
        return "F"

def df_names_fr():
    
    columns_order = ['gender', 'name', 'year', 'births']
    
    french_data = pd.read_csv(r"./data/french_name/nat2021.csv",sep = ';',
                              header=None, encoding = 'unicode_escape',
                              names = columns_order , keep_default_na = False)
    
    
    
    french_data = french_data.dropna()
    #print ("Step 1",french_data)
    # - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
    
    #french_data = french_data.rename({'preusuel': 'name', 'annais': 'year', 'sexe':'gender', 'nombre': 'births'}, axis='columns')


    new_order = ["year", "name", "gender", "births"]
    french_data = french_data [ new_order]

    #print ("Step 2",french_data)


   


    french_data['gender'] = french_data ['gender'].apply(convert_gender_label)
    #print ("Step 3",french_data)

    #  - Seuls les prénoms de 2 caractères et plus sont conservés
    french_data = remove_short_name(df=french_data, min_length=2)
    #print ("Step 4",french_data)

    #  - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
    #french_data = column_capitalize(french_data, col="name")
    french_data['name'] = french_data["name"].str.title()
    #print (french_data)

    #french_data['year'].unique() . I used this  to find out if there is any year that is not number
    #  - Les lignes avec des données inutilisables doivent être supprimées

    none_accepted_year_format = ['XXXX'] 
    none_accepted_name_format = ['Inconnu', 'Inconnue','_PRENOMS_RARES','_prenoms_rares'] 
    #none_accepted_name_format = ['_PRENOMS_RARES'] 
    damaged_year_data = french_data['year'].isin(none_accepted_year_format) #
    damaged_name_data = french_data['name'].isin(none_accepted_name_format)
    french_data = french_data [~damaged_year_data]
    #print ("Step 5",french_data)
    french_data = french_data [~damaged_name_data]
    #print (french_data)
    
    #print ("Step 6",french_data)




    #  - Mêmes dtypes pour les colonnes
    try:
        french_data ['year'] = pd.to_datetime (french_data ['year'] , format='%Y')
        
        print ("Step 7",french_data)
    except:
        print ("Error in converting year to Date time")
        
    try: 
        french_data['births'] = pd.to_numeric(french_data['births'] )
        print ("Step 8",french_data)
    except:
        print ("Error in converting births to numeric")




    #Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
    french_data = french_data.sort_values(by=[ 'gender', 'births','name','year'],
                                ascending=[False, False, True,False])

    # - L'index du DataFrame doit aller de 0 à N-1
    french_data =french_data.reset_index()
    french_data = french_data.drop(['index'], axis=1)
    return french_data

french_names = df_names_fr()

/tmp/ipykernel_224879/3301415410.py:14: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  french_data = pd.read_csv(r"./data/french_name/nat2021.csv",sep = ';',
/tmp/ipykernel_224879/3301415410.py:58: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  french_data = french_data [~damaged_name_data]


Error in converting year to Date time
Error in converting births to numeric


In [120]:

french_names.loc[french_names.name.str.contains(r"^[A-Z]+$")].empty


True

In [121]:
french_names.loc[french_names.name.str.contains(r"-[a-z]+$")].empty

True

In [122]:
len(french_names)- 648330

245

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [ ]:
# taux de change


us_dollar_data_file = r'./data/us_doller_data/Webstat_Export_20220928.csv'

us_dollar_data = pd.read_csv(us_dollar_data_file, sep=';', on_bad_lines='skip')
us_dollar_data = us_dollar_data.iloc[5:]

us_dollar_data.head(20)




,Titre :,Cours de l'or contre euro - 1er fixing à Londres,Cours de l'or contre dollar - 1er fixing à Londres
5,27/09/2022,"1696,76","1632,6"
6,26/09/2022,"1700,86",1647
7,25/09/2022,-,-
8,24/09/2022,-,-
9,23/09/2022,"1703,48","1661,45"
10,22/09/2022,"1692,92","1671,85"
11,21/09/2022,"1687,68","1674,45"
12,20/09/2022,"1665,56","1667,9"
13,19/09/2022,-,-
14,18/09/2022,-,-


In [ ]:
us_dollar_data.iloc[:, 0] = pd.to_datetime(us_dollar_data.iloc[:, 0])


/tmp/ipykernel_70303/4189371360.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  us_dollar_data.iloc[:, 0] = pd.to_datetime(us_dollar_data.iloc[:, 0])
/tmp/ipykernel_70303/4189371360.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  us_dollar_data.iloc[:, 0] = pd.to_datetime(us_dollar_data.iloc[:, 0])


In [128]:
import datetime
def read_curency_data():

    us_dollar_data_file = r'./data/us_doller_data/Webstat_Export_20220928.csv'

    us_dollar_data = pd.read_csv(us_dollar_data_file, sep=';', on_bad_lines='skip')
    us_dollar_data = us_dollar_data.iloc[5:]

    us_dollar_data.head(20)

    #us_dollar_data = us_dollar_data.dropna()

    us_dollar_data['date'] = us_dollar_data.iloc[:, 0]
    us_dollar_data['date'] =  pd.to_datetime(us_dollar_data['date'],  format='%d/%m/%Y')

    us_dollar_data['EUR'] = us_dollar_data.iloc[:, 1]
    us_dollar_data['EUR'] =us_dollar_data['EUR'].apply(lambda x: x.replace(',', '.'))
    us_dollar_data['EUR'] =us_dollar_data['EUR'].apply(lambda x: x.replace('-', ''))
    us_dollar_data['EUR'] = pd.to_numeric(us_dollar_data['EUR'])

    us_dollar_data['USD'] = us_dollar_data.iloc[:, 2]
    us_dollar_data['USD'] =us_dollar_data['USD'].apply(lambda x: x.replace(',', '.'))
    us_dollar_data['USD'] =us_dollar_data['USD'].apply(lambda x: x.replace('-', ''))
    us_dollar_data['USD'] = pd.to_numeric(us_dollar_data['USD'])

    us_dollar_data = us_dollar_data[['date', 'EUR', 'USD']]
    us_dollar_data = us_dollar_data.dropna()
    us_dollar_data =  us_dollar_data.reset_index(drop=True)
    
    return us_dollar_data


def df_taux_change(date , currency = 'EUR'):
    
    if isinstance(date, datetime):
        currency_data = read_curency_data()
        values = currency_data[us_dollar_data['date'] == date][currency].tolist()
        
    else:
        raise NameError('Your should use datatime format for the date')
    

data = read_curency_data()
data

,date,EUR,USD
0,2022-09-27,1696.76,1632.60
1,2022-09-26,1700.86,1647.00
2,2022-09-23,1703.48,1661.45
3,2022-09-22,1692.92,1671.85
4,2022-09-21,1687.68,1674.45
...,...,...,...
1662,2016-03-07,1156.96,1267.60
1663,2016-03-04,1158.67,1271.50
1664,2016-03-03,1141.48,1241.95
1665,2016-03-02,1131.53,1229.35


None


In [ ]:
from datetime import datetime
#a_date = datetime.strptime('2022-09-22', format="%d/%m/%Y")
a_date = datetime(2016, 3, 1)
type (a_date)
values = us_dollar_data[us_dollar_data['date'] == a_date]['EUR'].tolist()
values

NameError: name 'us_dollar_data' is not defined

In [ ]:
isinstance(a_date, datetime)

True

### Tests

In [25]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
    
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty) 

In [26]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [65]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... /tmp/ipykernel_224879/169755687.py:14: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  french_data = pd.read_csv(r"./data/french_name/nat2021.csv",sep = ';',


Step 1        gender            name    year  births
0        sexe        preusuel  annais  nombre
1           1  _PRENOMS_RARES    1900    1249
2           1  _PRENOMS_RARES    1901    1342
3           1  _PRENOMS_RARES    1902    1330
4           1  _PRENOMS_RARES    1903    1286
...       ...             ...     ...     ...
686534      2           ZYNEB    2018       5
686535      2           ZYNEB    2019       7
686536      2           ZYNEB    2020       8
686537      2           ZYNEB    2021       6
686538      2           ZYNEB    XXXX      19

[686539 rows x 4 columns]
Step 2           year            name gender  births
0       annais        preusuel   sexe  nombre
1         1900  _PRENOMS_RARES      1    1249
2         1901  _PRENOMS_RARES      1    1342
3         1902  _PRENOMS_RARES      1    1330
4         1903  _PRENOMS_RARES      1    1286
...        ...             ...    ...     ...
686534    2018           ZYNEB      2       5
686535    2019           ZYNEB      2  

/tmp/ipykernel_224879/169755687.py:52: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  french_data = french_data [~damaged_name_data]


Step 6           year      name gender  births
0       annais  Preusuel      F  nombre
126       2009     Aadam      F       4
127       2014     Aadam      F       3
128       2015     Aadam      F       3
129       2016     Aadam      F       4
...        ...       ...    ...     ...
686533    2017     Zyneb      F       6
686534    2018     Zyneb      F       5
686535    2019     Zyneb      F       7
686536    2020     Zyneb      F       8
686537    2021     Zyneb      F       6

[648331 rows x 4 columns]
Error in converting year to Date time
Error in converting births to numeric


FAIL
test_df_names_us (__main__.Session3Tests) ... 

Filed to read NationalReadMe.pdf
 cumlative leghnt is 2052781 and total leghnt is 2052781


ok
test_df_taux_change (__main__.Session3Tests) ... ERROR

ERROR: test_df_taux_change (__main__.Session3Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_224879/2219234773.py", line 32, in test_df_taux_change
    df = df_taux_change(["CHF", "GBP", "USD"])
NameError: name 'df_taux_change' is not defined

FAIL: test_df_names_fr (__main__.Session3Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_224879/2219234773.py", line 20, in test_df_names_fr
    self.assertEqual(len(df), 648330)
AssertionError: 648331 != 648330

----------------------------------------------------------------------
Ran 3 tests in 10.264s

FAILED (failures=1, errors=1)
